In [3]:
with open('./data/remap.word_id.topics.examples.json') as f:
    id2topics = json.loads(f.read())

FileNotFoundError: [Errno 2] No such file or directory: './data/remap.word_id.topics.examples.json'

In [ ]:
import json

In [ ]:
with open('./data/voa_sentences.json') as f:
    voa_sentences = json.loads(f.read())

voa_sample = {}
for k, v in voa_sentences.items():
    
    if len(v) > 10:
        v = list(set(v))
        v = sorted(v, key=lambda x: len(x.split()), reverse=True)
        sent = [line for line in v if len(line.split()) < 30 and len(line.split())>10][:10]
    else:
        sent = v
    voa_sample[k] = sent

In [ ]:
# with open('./data/voa_sample.json', 'w') as f:
#     f.write(json.dumps(voa_sample))

In [ ]:
import json
from collections import defaultdict

In [ ]:
with open('./data/MED.json') as f:
    med = json.loads(f.read())

# with open('./data/evaluation_sample.json') as f:
#     sample = json.loads(f.read())
    
# with open('./data/voa_sentences.json') as f:
#     voa = json.loads(f.read())

In [ ]:
over = []

In [ ]:
evaluation = defaultdict(list)
for word in voa.keys():
    count = 0
    for sense in med[word]['noun']['SENSE']:
        examples = [line[0] for line in sense[-1]]
        evaluation[word].extend(examples)
        count += len(examples)
    if count > 10:
        over.append(word)

for word, value in evaluation.items():
    if len(value) < 10:
        evaluation[word] += sample[word][:10-len(evaluation[word])]  

In [ ]:
sense_sep = defaultdict(dict)
for word in over:
    for idx, sense in enumerate(med[word]['noun']['SENSE']):
        examples = [line[0] for line in sense[-1]]
        sense_sep[word][idx] = examples

In [ ]:
for word in sense_sep:
    print(word)
    evaluation[word] = []
    count = 10
    while count > 0:
        for idx, sents in sense_sep[word].items():
            if sents:
                sents = sorted(sents, key=lambda x:len(x.split()), reverse=True)
                evaluation[word].append(sents[0])
                sense_sep[word][idx].pop(0)
                count -= 1
                if count == 0:
                    break

In [ ]:
for k, v in evaluation.items():
    print(k)
    print(len(v))

In [ ]:
with open('../data/0.0_top3_brt_map_cam.jsonl') as f:
    data = [json.loads(line) for line in f.readlines()]

for line in data:
    if 'cone.noun' in line['word_id']:
        print(line)
        print()

In [ ]:
with open('./data/med_voa_sample.json', 'w') as f:
    f.write(json.dumps(evaluation))

In [ ]:
with open('./data/med2cam_sense.noun.json') as f:
    med2cam = json.loads(f.read())

In [ ]:
for word in med2cam:
    for sense in med2cam[word]:
        examples = sense2examples[sense['med_def']]
        sense['examples'] = examples

In [ ]:
sense2examples = {}
for word in med:
    if 'noun' in med[word]:
        for sense in med[word]['noun']['SENSE']:
            en_sense = sense[0]
            examples = [line[0] for line in sense[-1]]
            sense2examples[en_sense] = examples

### From med sentences + voa sentence

In [7]:
import json
import random

In [8]:
with open('./data/med_sentences.json') as f:
    med = json.loads(f.read())
    
with open('./data/voa_sentences.json') as f:
    voa = json.loads(f.read())

In [18]:
evaluation_data = {}
voa_left_sent = {}

for word, value in med.items():
    print(word)
    examples = [sent[0] for data in value 
                for sent in data['examples']]
    evaluation_data[word] = examples
    if len(examples) < 30:
        left = 30 - len(examples)
        sentences = voa[word]
        if left > len(sentences):
            evaluation_data[word].extend(sentences)
        else:
            sentences = [line.replace('\n', '') for line in voa[word]
                        if len(line.split()) < 30]
            voa_left_sent[word] = sentences
            extend_sents = random.sample(sentences, left)
            evaluation_data[word].extend(extend_sents)
            for sent in extend_sents:
                voa_left_sent[word].remove(sent)
                
for word in voa:
    if word not in med:
        print(word)
        if len(voa[word]) < 30:
            sents = voa[word]
        else:
            sentences = [line.replace('\n', '') for line in voa[word]
                        if len(line.split()) < 30]
            voa_left_sent[word] = sentences
            
            sents = random.sample(sentences, 30)
            for sent in sents:
                voa_left_sent[word].remove(sent)
        evaluation_data[word] = sents

count = 0
thirty = 0
for k, v in evaluation_data.items():
    count += len(v)
    if len(v) == 30:
        thirty += 1
left = (300 - count) 
avg = left // thirty

left_count_map = {}
for k, v in evaluation_data.items():
    if len(v) == 30:
        left_count_map[k] = avg

number = left % thirty
while number > 0:
    for word in left_count_map:
        left_count_map[word] += 1
    number -= len(left_count_map)

for word, num in left_count_map.items():
    sentences = voa_left_sent[word]
    extend_sents = random.sample(sentences, num)
    evaluation_data[word].extend(extend_sents)

taste
issue
interest
star
duty
sentence
cone
bow
slug
bank
mole
bass


In [19]:
count = 0
thirty = 0
for k, v in evaluation_data.items():
    count += len(v)
count

305

In [20]:
evaluation_data.keys()

dict_keys(['taste', 'issue', 'interest', 'star', 'duty', 'sentence', 'cone', 'bow', 'slug', 'bank', 'mole', 'bass'])

In [21]:
with open('./data/300_sentences.json', 'w') as f:
    f.write(json.dumps(evaluation_data))